# How do I _create_ and _start_ a task?
### Overview
We are getting into advanced techniques here and will need to leverage a few other cookbooks. You will need to add an app and some files in your project, then it is easy to start one. 

### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You understand how to <a href="apps_listAll.ipynb" target="_blank">list</a> apps within one of your projects (we will just use that call directly here). This is a **great place** to get the **app_id** you will need in this recipe.
 5. You have at least one app in your project, maybe from <a href="apps_copyFromPublicApps.ipynb" target="_blank">copying one</a>
 6. You may want to review how to <a href="apps_detailOne.ipynb" target="_blank"> get details </a> of your app (we will assume you do, and pass the appropriate inputs).

 
### WARNING
This will burn through some processing credits (**about \$0.50**). You can create _DRAFT_ tasks to just see how it works, swap the commenting in **Build and run tasks** to only run: 
```python
# task created in DRAFT state
task = api.tasks.create(name=task_name, project=my_project.id, app=single_app.id, inputs=inputs, run=False)       

# task created and RUN immediately
task = api.tasks.create(name=task_name, project=my_project.id, app=single_app.id, inputs=inputs, run=True)       
```

## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# User input: specify platform {cgc, sbg}
prof = 'sbg'

config_config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_config_file)

## Find your project 
First, we identify an **interesting project** (by _name_) by searching though all of our projects<sup>1</sup>

In [ ]:
# [USER INPUT] Set project name:
project_name = 'Michael Diamond'     # project to copy app into
a_name = 'SBG FASTA Indices'


# FIND my project
my_project = [p for p in api.projects.query(limit=100).all() \
              if p.name == project_name]                         
if not my_project:
    print('Target project (%s) not found, check spelling' % project_name)
    raise KeyboardInterrupt
else:
    my_project = my_project[0]

## Copy an app to use for the task
Next, we find an **interesting app** (by _name_), again by searching though all of the apps _within_ that project<sup>1</sup>. Note, we are reusing my_project from above.)

<sup>1</sup> A _cleaner_ way to do this would be to identify by project and app **id**. Stay tuned for updates.

In [ ]:
# Apps already in my project
my_apps = api.apps.query(project = my_project.id, limit=100)

# Apps in the Public Reference
my_app_source = [a for a in api.apps.query(visibility='public', limit=100).all() \
                 if a.name == a_name]
if not my_app_source:
    print('App (%s) does not exist in Public Reference Apps' % (a_name))
    raise KeyboardInterrupt
else:
    my_app_source = my_app_source[0]

# Make sure NOT to copy the same app 2x, this angers the Platform considerably
duplicate_app = [a for a in my_apps.all() if a.name == my_app_source.name]

if duplicate_app:
    print('App already exists in second project, using that one')
    my_new_app = [a for a in api.apps.query(limit = 100, \
                   project = my_project.id).all() \
                   if a.name == a_name][0]
else:
    print('App (%s) does not exist in Project (%s); copying now' % \
          (a_name, my_project.name))
    
    my_new_app = my_app_source.copy(project = my_project.id, name = a_name)
        
    # re-list apps in target project to verify the copy worked
    my_apps = api.apps.query(project = my_project.id, limit=100)
    my_app_names = [a.name for a in my_apps.all()]
    
    if a_name in my_app_names:
        print('Sucessfully copied one app!')
    else:
        print('Something went wrong...')

## Copy a public reference file to use for an input
We will first find our _source\_project_ (the Public Reference Files), then list the files within the source project<sup>2</sup>, and copy a file from **_source\_project_ -> _my\_project_**.

<sup>2</sup> Files are only accessible **within** a project - here the Public Reference project (**warning** we may change this project name in the future).

In [ ]:
# [USER INPUT] Set file and Public Reference Project names here:
source_project_id = 'admin/sbg-public-data'        
f_name = 'ucsc.hg19.fasta'                                     # file to copy


# LIST all file names in target project
my_files = [f.name for f in api.files.query(limit = 100, project = my_project.id).all()]
# Find source file
source_file = [f for f in api.files.query(limit = 100, project = source_project_id).all() \
               if f.name == f_name]

# Make sure that file exists in Public Reference
if not source_file:
    print("File (%s) does not exist in Public Reference, please check spelling" \
         % (f_name))
    raise KeyboardInterrupt
else:
    source_file = source_file[0]
    
# Check if first file already exists in the target project
if source_file.name in my_files:
    print('File already exists in second project, using that one')
    my_new_file = [f for f in api.files.query(limit = 100, \
                   project = my_project.id).all() \
                   if f.name == source_file.name][0]
else:
    print('File (%s) does not exist in Project (%s); copying now' % \
          (source_file.name, my_project.id))

    my_new_file = source_file.copy(project = my_project.id, \
                                   name = source_file.name)

    # re-list files in target project to verify the copy worked
    my_files = [f.name for f in api.files.query(limit = 100, project = my_project.id).all()]
    
    if source_file.name in my_files:
        print('Sucessfully copied one file!')
    else:
        print('Something went wrong...')

## Create & start the task
Here we use the reference file and set one of the 11 optional configuration inputs. Note that input files are passed a _file_ (or a _list_ of _files_) while configuration parameters are passed just the values.

In [ ]:
# Task description
task_name = 'task created with task_create.ipynb'
inputs = {'reference':my_new_file} # 'fasta' is a 'File_Inputs'

# if your task has any 'Config_inputs' that can be specified by
#    {'id':value}      # value can be str, bool, float, etc depending on tool

# Create and RUN a task
my_task = api.tasks.create(name=task_name, project=my_project.id, \
                        app=my_new_app.id, inputs=inputs, run=True)

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/create-a-new-task) and [here](http://docs.sevenbridges.com/docs/perform-an-action-on-a-specific-task)